<a href="https://colab.research.google.com/github/Hyenni/BDAI-Training/blob/master/sparkLAB5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Persist an RDD

In [0]:
# Stub code to paste into the shell

sc.setLogLevel("WARN")

# Count web server log requests by user id
userReqs = sc.textFile("/loudacre/weblogs/*2.log")\
  .map(lambda line: line.split()) \
  .map(lambda words: (words[2],1)) \
  .reduceByKey(lambda v1,v2: v1+v2)

# Map account data to (userid,"lastname,firstname") pairs
accounts = sc.textFile("/loudacre/accounts/*")\
  .map(lambda s: s.split(',')) \
  .map(lambda values: (values[0],values[4] + ',' + values[3]))

# Join account names with request counts
accountHits=accounts.join(userReqs)\
  .map(lambda (userid,values): values) 


In [0]:
#  How many accounts had hitcount > 5? 
accountHits.filter(lambda (firstlast,hitcount): hitcount > 5).count()

# Persist the RDD
accountHits.persist()


PythonRDD[36] at RDD at PythonRDD.scala:43

In [0]:
# Rerun the job
accountHits.filter(lambda (firstlast,hitcount): hitcount > 5).count()

print accountHits.toDebugString()


(23) PythonRDD[36] at RDD at PythonRDD.scala:43 [Memory Serialized 1x Replicated]
 |        CachedPartitions: 23; MemorySize: 311.2 KB; ExternalBlockStoreSize: 0.0 B; DiskSize: 0.0 B
 |   MapPartitionsRDD[34] at mapPartitions at PythonRDD.scala:374 [Memory Serialized 1x Replicated]
 |   ShuffledRDD[33] at partitionBy at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]
 +-(23) PairwiseRDD[32] at join at <ipython-input-6-1a021ee00ccc>:12 [Memory Serialized 1x Replicated]
    |   PythonRDD[31] at join at <ipython-input-6-1a021ee00ccc>:12 [Memory Serialized 1x Replicated]
    |   UnionRDD[30] at union at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]
    |   PythonRDD[28] at RDD at PythonRDD.scala:43 [Memory Serialized 1x Replicated]
    |   /loudacre/accounts/* MapPartitionsRDD[27] at textFile at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]
    |   /loudacre/accounts/* HadoopRDD[26] at textFile at NativeMethodAccessorImpl.java:-2

In [0]:
from pyspark import StorageLevel
accountHits.unpersist()
accountHits.persist(StorageLevel.DISK_ONLY)

accountHits.filter(lambda (firstlast,hitcount): hitcount > 5).count()

5872

### in Memory vs on Disk

![persist](https://user-images.githubusercontent.com/33708512/56344343-227e5880-61f8-11e9-9c4d-0695d68ff1f2.png)
